<a href="https://colab.research.google.com/github/Twinkle0799/Predicton-of-House-prices-using-tensorflow/blob/master/Tensorflow%20house%20price%20predicting%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTING LIBRARIES**

In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
#from utils import *
import sys
sys.path.append('PATH_TO_TENSORFLOW_OBJECT_DETECTION_FOLDER')
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

%matplotlib inline
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

print('Libraries Imported')

Libraries Imported
